In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-06-30')
end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow_64_359350"

#infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [31]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

# clean user names
long_jobs['user'] = long_jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(long_jobs, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 336787 long jobs in the Alina's interval
after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 359350 records


In [32]:
jobs_to_nodes_whole_data.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
1,17,498458.node129,16,0,0,14680064


In [33]:

long_jobs.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,mic_req,real_pow,ran_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes
job_id,,,,,,,,,,,,,,,,,,,,,
205722,1006691.node129,sub_Tio2_2x2x1_,parallel,2014-08-13 00:01:21,2014-08-13 00:03:14,2014-08-13 03:03:48,mpalummo,1,16,0,...,NaN,245.510838,True,0.0,1,0,0,2166,15,1


In [34]:
queues = long_jobs['queue'].unique()

In [35]:
nodes_labels = ['node_%s'%(x) for x in range(1, 64)]
nodes_counters = ['node_count_%s'%(x) for x in range(1, 64)]

#nodes_labels + nodes_counters

We want to obtain 5 dataframes:
- node
- user
- user_node
- user_job
- user_job_node

In [83]:
# node dataframe
index_node = jobs_to_nodes_whole_data.groupby(['node_id']).size().reset_index(name='counts')
columns_node = ['node_id', 'counts', 'avg']
df_node = pd.DataFrame(index_node, index=range(0, index_node.shape[0]), columns=columns_node)
df_node.head(3)

,node_id,counts,avg
0,1,1875,NaN


In [84]:
# user dataframe
index_user = long_jobs.groupby(['user']).size().reset_index(name='counts')
columns_user = ['user', 'counts', 'avg']
df_user = pd.DataFrame(index_user, index=range(0, index_user.shape[0]), columns=columns_user)
df_user.head(3)

,user,counts,avg
0,a07smr01,2239,NaN


In [87]:
# user job dataframe
index_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).size().reset_index(name='counts')
columns_user_job = ['user', 'node_id', 'counts', 'avg']
df_user_node = pd.DataFrame(index_user_node, index=range(0, index_user_node.shape[0]), columns=columns_user_job)
df_user_node.head(3)

,user,node_id,counts,avg
0,a07smr01,1,6,NaN


In [88]:
# user dataframe
index_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).size().reset_index(name='counts')
columns_user_job = ['user', 'job_name', 'counts', 'avg']
df_user_job = pd.DataFrame(index_user_job, index=range(0, index_user_job.shape[0]), columns=columns_user_job)
df_user_job.head(3)

,user,job_name,counts,avg
0,a07smr01,A2REPO,109,NaN


In [89]:
index_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).size().reset_index(name='counts')
columns_user_job_node = ['user', 'job_name', 'node_id', 'counts', 'avg']
df_user_job_node = pd.DataFrame(index_user_job_node, index=range(0, index_user_job_node.shape[0]), columns=columns_user_job_node)
df_user_job_node.head(3)

,user,job_name,node_id,counts,avg
0,a07smr01,A2REPO,7,1,NaN


In [91]:
print(df_node.shape[0])
print(df_user.shape[0])
print(df_user_node.shape[0])
print(df_user_job.shape[0])
print(df_user_job_node.shape[0])

63
277
5527
15505
55185


In [129]:
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] * merged_jobs_to_nodes['ncpus'] / merged_jobs_to_nodes['cpu_req'] / merged_jobs_to_nodes['ncpus']

In [134]:
# user_job_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job_node.shape[0])
df_user_job_node.head(5)

55185


,user,job_name,node_id,mean_core,count
0,a07smr01,A2REPO,7,12.412156,1
1,a07smr01,A2REPO,33,5.859882,4
2,a07smr01,A2REPO,35,5.483522,79
3,a07smr01,A2REPO,46,2.406068,24
4,a07smr01,A2REPO,49,0.000000,1


In [135]:
# user_job_node
df_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_job.shape[0])
df_user_job.head(5)

15505


,user,job_name,mean_core,count
0,a07smr01,A2REPO,4.832987,109
1,a07smr01,ARCHIVE,10.621560,2
2,a07smr01,ARKI,4.598149,125
3,a07smr01,INIT,5.044202,137
4,a07smr01,LM28_OPE,20.379112,1466


In [136]:
# user_node
df_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user_node.shape[0])
df_user_node.head(5)

5527


,user,node_id,mean_core,count
0,a07smr01,1,13.377256,6
1,a07smr01,2,13.390850,6
2,a07smr01,6,18.405537,4
3,a07smr01,7,28.234849,3
4,a07smr01,33,6.901498,66


In [137]:
# user
df_user = merged_jobs_to_nodes.groupby(['user']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_user.shape[0])
df_user.head(5)

277


,user,mean_core,count
0,a07smr01,13.636318,8509
1,a08tra01,35.482267,37
2,a08tra11,138.092955,4
3,a08tra30,16.518865,2
4,a08tra35,30.627245,12


In [139]:
# node
df_node = merged_jobs_to_nodes.groupby(['node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print(df_node.shape[0])
df_node

63


,node_id,mean_core,count
0,1,9.940877,1648
1,2,9.330023,239
2,3,15.076136,1327
3,4,15.428986,1421
4,5,20.054974,482
5,6,17.429749,1172
6,7,14.690999,1349
7,8,15.699614,1153
8,9,14.420087,703
9,10,14.898309,1132


In [12]:
i = 0

user_names = long_jobs['user'].unique()

for user_name in user_names:
    user_jobs = long_jobs[long_jobs['user'] == user_name]
    
    user_jobs_names = user_jobs['job_name'].unique()
    
    for job_name in user_jobs_names:
        user_jobs_name = user_jobs[user_jobs['job_name'] == job_name]

        for job_index, job_row in user_jobs_name.iterrows():
            merged_job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_row['job_id_string']]
            print(job_row['job_name'])
       
    

sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_2x2x1_
sub_Tio2_1x1x1_
sub_Tio2_1x1x1_
sub_ephR_S
sub_eph
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new_STEP3
sub_new

KeyboardInterrupt: 